# Transient dynamics 

In [ ]:
include("../age_structure/age_structure_model.jl")
include("../age_structure/fecundity_at_age.jl")
include("i_state_dsn_pedigree_triat.jl")
using LinearAlgebra
using Plots
using Roots
using Distributions

┌ Info: Precompiling DSP [717857b8-e6f2-59f4-9121-6e50c889abd2]
└ @ Base loading.jl:1278


In [ ]:
# single paramters 
A_max = 100 # max age
A_mature = 0 # age at maturity 
f_infty = 10.0^6 # maximum fecundity 
k = 0.1 # growth rate 
a0 = 10.0 # growth paramter
s = 0.95 # survival rate


# vectorized paramters
fecundity = broadcast(x -> fecundity_at_age.isometric_LVB(x,f_infty,k,a0), 1:A_max)
survival = repeat([s],A_max)

A = age_structure_model.build_age_matrix(A_max,fecundity,survival)
print(" ")